In [ ]:
# Import libaries: 

#Data manipulation
import pandas as pd
import numpy as np

#Visualization
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import PolyCollection

# Change the parameters
plt.rcParams.update({'mathtext.default':  'regular' })
sns.set(context='paper',style="whitegrid",font='Times New Roman',font_scale=1.2)

## Visualization of the experimental data

In [ ]:
# The name of the files are given by these numbers
numbers = ['9','12','15','18','21','24']

# The known angles of the experiment are in this list
angles =[5,15,25,35,45,55]

# Function that later will be applied into the dataframe
def ang(x):
    x = str(x)
    return f'Angle: '+ x

# Generate a list of dataframes (looking foward to concatenate)
dataframes=[]
# A for loop to generate the dataframes from files
for i,let in enumerate(numbers):
    #Read files
    with open(f'./SiO {let}(Si).xy', 'r') as fout:
        value=fout.read()
    #Clean data a little
    vals = value.split('#')[-1].split('\n')
    while '' in vals:
        vals.remove('')
    #Generate the dataframe
    data = pd.DataFrame([row.split('  ') for row in vals], columns=[f'bind',f'exp']).astype('float')
    #Add a new column to indicate that there is an angle dependence
    data['Angle']=90-angles[i]*np.ones(data.shape[0])
    data['Angle']=data['Angle'].apply(ang)
    #Integrate to the list
    dataframes.append(data)
    
#Generate a huge dataframe with all the information
all_data = pd.concat(dataframes, axis=0)

In [ ]:
# Select a palette: 
COLORS=[(0,0,1,.2),(0,0,1,.3),(0,0,1,.4),(0,0,1,.5),(0,0,1,.6),(0,0,1,.7)]
COLORS2=sns.cubehelix_palette(rot=-.4)

# Plot with seaborn lineplot, add a hue. 

plt.figure(figsize=(15,10))
sns.scatterplot(all_data.bind, all_data.exp, hue=all_data.Angle, palette=COLORS2,legend=False)
sns.lineplot(all_data.bind, all_data.exp, hue=all_data.Angle, palette=COLORS2)
plt.xlim(108, 96)

plt.xlabel('Binding Energy (eV)')
plt.ylabel('Intensity (Counts/s)')
plt.suptitle('Si-SiO$_2$ Experimental Data', fontsize=30)
plt.grid(which='both', c='#D4D4D4')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,10))
ax = fig.gca(projection='3d')


# Add vertices for a polygon figure which is a list of touples (X,y)
verts2=[]
for data in dataframes:
    value_verts=[]
    for i in range(len(list(data.exp))):
        z=(float(list(data.bind)[i]),float(list(data.exp)[i]))
        value_verts.append(z)
    verts2.append(value_verts)

# Generate a collection
poly = PolyCollection(verts2,facecolors=COLORS2)


# Determine the positions on z axis. 
zs=90-np.array(angles)

# Plot
ax.add_collection3d(poly, zs=zs, zdir='y')
ax.set_xlabel('Binding Energy (eV)')
ax.set_xlim3d(111, 95)
ax.set_ylabel('Angle (degrees)')
ax.set_ylim3d(30, 90)
ax.set_zlabel('Intensity (Counts/s)')
ax.set_zlim3d(0, 4200)
plt.show()

## Visualization of data after processing in Aanalyzer

In [ ]:
# List of dataframes with fit curves from the obtained parameters
cal_dataframes = []
for i,let in enumerate(numbers):
    cal_data=pd.read_csv(f'./Angulaes_sin_ajuste/SiO {let}(Si).fit', sep='\t')[['x','yCal']].astype('float')
    cal_data['Angle'] = 90-angles[i]*np.ones(cal_data.shape[0])
    cal_data['Angle'] = cal_data['Angle'].apply(ang)
    cal_dataframes.append(cal_data)

# Concatenate the dataframes
all_cal_data = pd.concat(cal_dataframes, axis=0)

#Generate a visualization
plt.figure(figsize=(15,10))
sns.lineplot(all_cal_data.x, all_cal_data.yCal, hue=all_cal_data.Angle, palette=COLORS2)
plt.xlim(108, 96)
plt.xlabel('Binding Energy (eV)')
plt.ylabel('Intensity (Counts/s)')
plt.suptitle('Si-SiO$_2$', fontsize=30)
plt.grid(which='both', c='#D4D4D4')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,10))
ax = fig.gca(projection='3d')

verts3=[]
for data in cal_dataframes:
    value_verts=[]
    for i in range(len(list(data.x))):
        z=(float(list(data.x)[i]),float(list(data.yCal)[i]))
        value_verts.append(z)
    verts3.append(value_verts)
    
zs=90-np.array(angles)
    
poly = PolyCollection(verts3,facecolors=COLORS2)
ax.add_collection3d(poly, zs=zs, zdir='y')
ax.set_xlabel('Binding Energy (eV)')
ax.set_xlim3d(111, 95)
ax.set_ylabel('Angle (degrees)')
ax.set_ylim3d(30, 90)
ax.set_zlabel('Intensity (Counts/s)')
ax.set_zlim3d(0, 4200)
plt.show()

# Results table and area comparison

In [ ]:
# Obtain table of results to generate dataframes:

table_areas = pd.read_csv('./Angulaes_sin_ajuste/areaTable.tbl', sep='\t')

cols = list(filter(lambda x: 'Unnamed' not in x, list(table_areas.columns)))
table_areas = table_areas[cols].head(6)
table_areas.columns = ['angle','x','p0','p1','p2','p3','tot_area']

# Knowing how the data was processed is necessary to obtain this information
table_areas['p_si']=table_areas.p0+table_areas.p2
table_areas['p_si_o2']=table_areas.p1+table_areas.p3
table_areas['R']=table_areas.p_si_o2/table_areas.p_si
table_areas

In [ ]:
# Plotting the area of the peaks with respect to the complementary angle

sns.lineplot(table_areas.x,table_areas.p_si_o2,label='SiO$_2$')
sns.scatterplot(table_areas.x,table_areas.p_si_o2)

sns.lineplot(table_areas.x,table_areas.p_si,label='Si')
sns.scatterplot(table_areas.x,table_areas.p_si)


plt.xlabel('Complementary Angle')
plt.ylabel('Area')
plt.title('Area for Si and SiO$_2$')

In [ ]:
# Obtaining the relation between the areas 

sns.lineplot(table_areas.x,table_areas.R)
sns.scatterplot(table_areas.x,table_areas.R)
plt.xlabel('Complementary Angle')
plt.ylabel('R: $I_{SiO_2}/I_{Si}$')
plt.title('R, Relation between areas')
plt.show()

## Analysis of the infinite samples: Si and SiO2

In [ ]:
# The name of the files are given by these numbers
numbers = ['9','12','15','18','21','24']

# The known angles of the experiment are in this list
angles =[5,15,25,35,45,55]

# Function that later will be applied into the dataframe
def ang(x):
    x = str(x)
    return f'Angle: '+ x

# Generate a list of dataframes (looking foward to concatenate)
dataframes=[]
# A for loop to generate the dataframes from files
for i,let in enumerate(numbers):
    #Read files
    with open(f'./Silicio2p/OxidoInfinito/OxidoInfinito_single {let}.xy', 'r') as fout:
        value=fout.read()
    #Clean data a little
    vals = value.split('#')[-1].split('\n')
    while '' in vals:
        vals.remove('')
    #Generate the dataframe
    data = pd.DataFrame([row.split('  ') for row in vals], columns=[f'bind',f'exp']).astype('float')
    #Add a new column to indicate that there is an angle dependence
    data['Angle']=90-angles[i]*np.ones(data.shape[0])
    data['Angle']=data['Angle'].apply(ang)
    #Integrate to the list
    dataframes.append(data)
    
#Generate a huge dataframe with all the information
all_data = pd.concat(dataframes, axis=0)


# Plot with seaborn lineplot, add a hue. 

plt.figure(figsize=(15,10))
sns.scatterplot(all_data.bind, all_data.exp, hue=all_data.Angle, palette=COLORS2,legend=False)
sns.lineplot(all_data.bind, all_data.exp, hue=all_data.Angle, palette=COLORS2)
plt.xlim(108, 96)

plt.xlabel('Binding Energy (eV)')
plt.ylabel('Intensity (Counts/s)')
plt.suptitle('SiO$_2$ ($\\infty$) Experimental data', fontsize=30)
plt.grid(which='both', c='#D4D4D4')
plt.show()

In [ ]:
# The name of the files are given by these numbers
numbers = ['9','12','15','18','21','24']

# The known angles of the experiment are in this list
angles =[5,15,25,35,45,55]

# Function that later will be applied into the dataframe
def ang(x):
    x = str(x)
    return f'Angle: '+ x

# Generate a list of dataframes (looking foward to concatenate)
dataframes=[]
# A for loop to generate the dataframes from files
for i,let in enumerate(angles):
    #Read files
    #with open(f'./Silicio2p/SilicioInfinito/Si2p {let}.txt', 'r') as fout:
    #    value=fout.read()
    #Generate the dataframe
    
    #data = pd.DataFrame(value, columns=[f'bind',f'exp']).astype('float')
    data=pd.read_csv(f'./Silicio2p/SilicioInfinito/Si2p {let}.txt',sep='\s+', header=None, dtype=float)
    data.columns=[f'bind',f'exp']
    #Add a new column to indicate that there is an angle dependence
    data['Angle']=90-angles[i]*np.ones(data.shape[0])
    data['Angle']=data['Angle'].apply(ang)
    #Integrate to the list
    dataframes.append(data)
    
#Generate a huge dataframe with all the information
all_data = pd.concat(dataframes, axis=0)


# Plot with seaborn lineplot, add a hue. 

plt.figure(figsize=(15,10))
sns.scatterplot(all_data.bind, all_data.exp, hue=all_data.Angle, palette=COLORS2,legend=False)
sns.lineplot(all_data.bind, all_data.exp, hue=all_data.Angle, palette=COLORS2)
plt.xlim(108, 96)

plt.xlabel('Binding Energy (eV)')
plt.ylabel('Intensity (Counts/s)')
plt.suptitle('Si ($\\infty$) Experimental data', fontsize=30)
plt.grid(which='both', c='#D4D4D4')
plt.show()

In [ ]:
cal_dataframes = []
for i,let in enumerate(numbers):
    cal_data=pd.read_csv(f'./Silicio2p/OxidoInfinito/OxidoInfinito_single {let}.fit', sep='\t')[['x','yCal']].astype('float')
    cal_data['Angle'] = 90-angles[i]*np.ones(cal_data.shape[0])
    cal_data['Angle'] = cal_data['Angle'].apply(ang)
    cal_dataframes.append(cal_data)
all_cal_data = pd.concat(cal_dataframes, axis=0)
plt.figure(figsize=(15,10))
sns.lineplot(all_cal_data.x, all_cal_data.yCal, hue=all_cal_data.Angle, palette=COLORS2)
plt.xlim(max(all_cal_data.x), min(all_cal_data.x))

plt.xlabel('Binding Energy (eV)')
plt.ylabel('Intensity (Counts/s)')
plt.suptitle('SiO$_2$ ($\\infty$) ', fontsize=30)
plt.grid(which='both', c='#D4D4D4')
plt.show()

In [ ]:
cal_dataframes = []
for i,let in enumerate(angles):
    cal_data=pd.read_csv(f'./Silicio2p/SilicioInfinito/Si2p {let}.fit', sep='\t')[['x','yCal']].astype('float')
    cal_data['Angle'] = 90-angles[i]*np.ones(cal_data.shape[0])
    cal_data['Angle'] = cal_data['Angle'].apply(ang)
    cal_dataframes.append(cal_data)
all_cal_data = pd.concat(cal_dataframes, axis=0)
plt.figure(figsize=(15,10))
sns.lineplot(all_cal_data.x, all_cal_data.yCal, hue=all_cal_data.Angle, palette=COLORS2)
plt.xlim(max(all_cal_data.x), min(all_cal_data.x))

plt.xlabel('Binding Energy (eV)')
plt.ylabel('Intensity (Counts/s)')
plt.suptitle('Si ($\\infty$) ', fontsize=30)
plt.grid(which='both', c='#D4D4D4')
plt.show()

In [ ]:
table_areas_ox_inf = pd.read_csv('./Silicio2p/OxidoInfinito/areaTable_oxido_infinito.tbl', sep='\t')

cols = list(filter(lambda x: 'Unnamed' not in x, list(table_areas_ox_inf.columns)))
table_areas_ox_inf = table_areas_ox_inf[cols].head(6)
table_areas_ox_inf.columns = ['angle','x','p0','p1','tot_area']
table_areas_ox_inf['p_si']=table_areas_ox_inf.p0+table_areas_ox_inf.p1
table_areas_ox_inf.x=[85,75,65,55,45,35]

table_areas_si_inf = pd.read_csv('./Silicio2p/SilicioInfinito/areaTable_silicio_infinito.tbl', sep='\t')

cols = list(filter(lambda x: 'Unnamed' not in x, list(table_areas_si_inf.columns)))
table_areas_si_inf = table_areas_si_inf[cols].head(6)
table_areas_si_inf.columns = ['angle','x','p0','p1','tot_area']
table_areas_si_inf['p_si']=table_areas_si_inf.p0+table_areas_si_inf.p1
table_areas_si_inf


sns.lineplot(table_areas_ox_inf.x,table_areas_ox_inf.tot_area,label='SiO$_2$')
sns.scatterplot(table_areas_ox_inf.x,table_areas_ox_inf.tot_area)



sns.lineplot(table_areas_si_inf.x,table_areas_si_inf.tot_area, label='Si')
sns.scatterplot(table_areas_si_inf.x,table_areas_si_inf.tot_area)

plt.xlabel('Complementary Angle')
plt.ylabel('Area')
plt.title('Area of Infinite Si and SiO$_2$ samples')
plt.show()

In [ ]:
sns.scatterplot(table_areas_si_inf.x,table_areas_ox_inf.tot_area/table_areas_si_inf.tot_area)
sns.scatterplot(table_areas_si_inf.x,table_areas_ox_inf.tot_area/table_areas_si_inf.tot_area)

sns.lineplot(table_areas_si_inf.x,table_areas_ox_inf.tot_area/table_areas_si_inf.tot_area)
sns.lineplot(table_areas_si_inf.x,table_areas_ox_inf.tot_area/table_areas_si_inf.tot_area)
plt.xlabel('Angle')
plt.ylabel('R: $I_{SiO2 (inf)} / I_{Si (inf)}$')
plt.title('R, Relation between areas of SiO$_2$ and Si ')
plt.show()

In [ ]:
#Create table with useful data
end_table = pd.DataFrame([table_areas_si_inf.x,
                          pd.Series(angles),
                          table_areas.p_si_o2,
                          table_areas.p_si,
                          table_areas_ox_inf.tot_area,
                          table_areas_si_inf.tot_area,
                          table_areas_ox_inf.tot_area/table_areas_si_inf.tot_area,
                          table_areas.R])
end_table = end_table.T
end_table.columns = ['angle_c','angle','sio2_exp','si_exp','sio2_inf','si_inf','K', 'R']
end_table['ln 1+R/K']=np.log(1+end_table.R/end_table.K)
end_table['rad_c']=(end_table.angle_c)/180*np.pi
end_table['rad']=(end_table.angle)/180*np.pi
end_table['1/sin']=1/np.sin(end_table.rad)
end_table

In [ ]:
from scipy import stats

In [ ]:
x0=end_table['1/sin'][0:]
y0=end_table['ln 1+R/K'][0:]
slope0, intercept0, r_value, p_value, std_err = stats.linregress(x0,y0)

x1=end_table['1/sin'][1:]
y1=end_table['ln 1+R/K'][1:]
slope1, intercept1, r_value, p_value, std_err = stats.linregress(x1,y1)

x2=end_table['1/sin'][2:]
y2=end_table['ln 1+R/K'][2:]
slope2, intercept2, r_value, p_value, std_err = stats.linregress(x2,y2)

In [ ]:
plt.plot(end_table['1/sin'],end_table['ln 1+R/K'])
plt.scatter(end_table['1/sin'],end_table['ln 1+R/K'])

z0=np.linspace(-9,10)
z1=np.linspace(-1,4)
z2=np.linspace(.3,4)

a=slope0*z0+intercept0
b=slope1*z1+intercept1
c=slope2*z2+intercept2

plt.plot(z0,a,'--', c='r')
plt.plot(z1,b,'--', c='g')
plt.plot(z2,c,'--', c='orange')


plt.grid(True, which='both')
plt.xlabel('1/sin($\\theta$)')
plt.ylabel('ln (1+R/K)')

plt.axhline(y=0, color='k')
plt.axvline(x=0, color='k')

In [ ]:
plt.plot(end_table['1/sin'],end_table['ln 1+R/K'])
plt.scatter(end_table['1/sin'],end_table['ln 1+R/K'])

plt.plot(z0,a,'--', c='r')

plt.grid(True, which='both')
plt.xlabel('1/sin($\\theta$)')
plt.ylabel('ln (1+R/K)')

plt.axhline(y=0, color='k')
plt.axvline(x=0, color='k')

In [ ]:
plt.plot(end_table['1/sin'],end_table['ln 1+R/K'])
plt.scatter(end_table['1/sin'],end_table['ln 1+R/K'])

plt.plot(z1,b,'--', c='g')

plt.grid(True, which='both')
plt.xlabel('1/sin($\\theta$)')
plt.ylabel('ln (1+R/K)')

plt.axhline(y=0, color='k')
plt.axvline(x=0, color='k')

In [ ]:
plt.plot(end_table['1/sin'],end_table['ln 1+R/K'])
plt.scatter(end_table['1/sin'],end_table['ln 1+R/K'])

plt.plot(z2,c,'--', c='orange')

plt.grid(True, which='both')
plt.xlabel('1/sin($\\theta$)')
plt.ylabel('ln (1+R/K)')

plt.axhline(y=0, color='k')
plt.axvline(x=0, color='k')